# 1. Creació de l'estructura on es guardaran les dades
Es generen tres classes:
<p>1) Classe eleccions
<p>2) Classe municipi
<p>3) Classe partit
<p>La primera classe detalla l'any de les eleccions i conté una llista amb tots els partits polítics. 
<p>La segona classe detalla tots els atributs que conté cada partit polític com vots, el percentatge de vots respecte el total, majoria absoluta, majria relativa i empats.
<p>La tercera classe detalla tots els atributs que conté cada partit polític com vots, el percentatge de vots respecte el total i regidors.

In [1]:
class eleccions:
    # Inicialitzador de la classe
    def __init__(self, ano):
        self.ano = ano
        self.municipis = []

class municipi:
    # Inicialitzador de la classe
    def __init__(self, nom, url):
        self.__nom = nom
        self.__url = url
        self.partits = []
        self.__cens = 0
        self.__escrutat = '00,00%'
        self.__abstencio = 0
        self.__vots_comp = 0
        self.__vots_nuls = 0
        self.__vots_blanc = 0
    # Getters de cadascun dels atributs
    def getNom(self):
        return self.__nom
    def getUrl(self):
        return self.__url
    def getPartit(self,index):
        return self.partits[index]
    def getLengthPartits(self):
        return len(self.partits)
    def setPartit(self,partit):
        self.partits.append(partit)
    def setAtributs(self,cens,escrutat,abstencio,vots_comp,vots_nuls,vots_blanc):
        self.__cens = cens
        self.__escrutat = escrutat
        self.__abstencio = abstencio
        self.__vots_comp = vots_comp
        self.__vots_nuls = vots_nuls
        self.__vots_blanc = vots_blanc 
    def getAtributs(self):
        return print('Nom: ',self.__nom,' Cens: ',self.__cens,' Escrutat: ',self.__escrutat,\
                     'Abstencio: ',self.__abstencio,'Vots comptabilitats: ',self.__vots_comp,\
                     ' Vots nuls: ',self.__vots_nuls,' Vots blancs: ',self.__vots_blanc)

class partit:
    # Inicialitzador de la classe
    def __init__(self,nom,acronim,vots,percent,regidors):
        self.__nom = nom
        self.__acronim = acronim
        self.__vots = vots
        self.__percent = percent
        self.__regidors = regidors
    # Getters de cadascun dels atributs
    def getNom(self):
        return self.__nom
    def getAcronim(self):
        return self.__acronim
    def getVots(self):
        return self.__vots
    def getPercent(self):
        return self.__percent
    def getRegidors(self):
        return self.__regidors
    def getTot(self):
        return print('Nom: ',self.__acronim,' Vots: ',self.__vots,' %: ',self.__percent,\
                     ' Regidos: ',self.__regidors)

  

<p> A continuació es detallen les funcions que s'utilitzen per la manipulació i extraccio de les dades:
<p>1) llistaMunicipis: Funció que cerca el llistat de municipis de les eleccions a Catalunya.  

In [2]:
def llistaMunicipis(llista_municipis,llista_municipis_1,web):
    # INPUT: Directori web on cerca els municipis i llistes municipis buides
    # OUTPUT: Retorna llistes amb tots els municipis i la seva URL associades

    # Càrrega i obtenció de la pàgina web
    page = requests.get(web)
    soup = BeautifulSoup(page.content,"html.parser")
    
    counter = 1 #variable contador que indica a quina linia es treballa
    for tag in soup.find_all('li', class_ = 'list-group-item'):
        # Els municipis comencen al 47 i acaben al 993. (1-47 provincies i comarques de catalunya i del 994-final C.C.A.A).
        if counter >= 47 and counter <= 993: 
            # S'obté el nom de cada municipi i la seva URL
            nom = tag.a.string # S'accedeix al nom
            url =tag.a['href'] # S'accedeix a la url

            # S'afegueix el municipi a la llista de municipis
            llista_municipis.append(municipi(nom,url))
            llista_municipis_1.append(municipi(nom,url))
        #S'incrementa en 1 el contador
        counter +=1

<p> 2) lecturaMunicipis: Funció que extreu d'una taula HTML la informació relativa a cada partit polític corresponent a un municipi.

In [3]:
def lecturaMunicipis(tag_taula,municipi):
    # INPUT: tag_taula: tag amb la taula HTML amb els partits
    #        municipi: variable tipus municipi amb la llista de partits buida
    # OUTPUT: municipi modificada ambla llista de partits plena
    
    # llistes nom_partits i acronim partits buides
    noms_partits = []
    acronim_partits = []
    # Llista auxiliar valors buida
    valors = []
    
    # Extracció de nom i l'acrònim de cada partit
    #for i in soup.table.find_all('td',class_ ="linkeleccionsd3 atribpartido"):
    for i in tag_taula.find_all('td',class_ ="linkeleccionsd3 atribpartido"):
        noms_partits.append(i['title'])
        acronim_partits.append(i.findNext(text = True))


    # Extracció dels valors {vots,%,Regidors}, per cada partit->3 valors.
    for i in tag_taula.find_all('td',class_ ="linkeleccionsd3 text-right"):
        valors.append(i.findNext(text = True))

    # Es junta tot
    i = 0
    j = 0

    if len(noms_partits)*3 == len(valors):
        while i<len(valors):
            vots = valors[i]
            i +=1
            percent = valors[i]
            i +=1
            regidors = valors[i]
            i +=1
            partt = partit(noms_partits[j],acronim_partits[j],vots,percent,regidors)
            municipi.setPartit(partt)
            #print('Nom: ',acronim_partits[j],' Vots: ',vots,' %: ',percent,' Regidos: ',regidors)
            j +=1
        #print("Obtenció dades correcte")
    else:
        print("¡¡ERROR!! - Obtenció dades incorrecte")

<p> 3) lecturaAtributs: Funció que extreu d'una taula HTML els atribut corresponents a cada localitat. La informació que s'extreu és Cens, total escrutat (%), abstencions, vots comptabilitzats, vots nuls i vots en blanc.

In [4]:
def lecturaAtributs(tag,eleccions):
    # INPUT: tag: variable tipus tag que conté la taula amb les dades de cens, etc, del municipi
    #        eleccions: llista de classe municipi   
    # OUTPUT: Variable municipi actualitzada amb els atributs
    
    aux = tag.findNext('td')
    cens = aux.findNext('td')
    aux = cens.findNext('td')
    aux = aux.findNext('td')
    escrutat = aux.findNext('td')
    aux = escrutat.findNext('td')

    vots_comp = aux.findNext('td')
    vots_comp_percent = vots_comp.findNext('td')

    aux = vots_comp_percent.findNext('td')
    abs_comp = aux.findNext('td')
    abs_comp_percent = abs_comp.findNext('td')

    aux = abs_comp_percent.findNext('td')
    vots_nuls = aux.findNext('td')
    vots_nuls_percent = vots_nuls.findNext('td')

    aux = vots_nuls_percent.findNext('td')
    vots_blanc = aux.findNext('td')
    vots_blanc_percent = vots_blanc.findNext('td')
    
    # Es mostra per pantalla les dades extretes de cada municipi com a check point
    eleccions.setAtributs(cens.string,escrutat.string,abs_comp.string,vots_comp.string,\
                          vots_nuls.string,vots_blanc.string)

<p> 4) extraccioDades: Funció que llegeix la llista de municipis i extreu tota la informació de les eleccions de 2019 i 2015 per municipis.

In [5]:
def extraccioDades(elec_2019,elec_2015):
    # INPUT: elec_20XX: llista de classe municipi amb noms i urls de cadascun ()
    # OUTPUT: elec_20XX: modificada amb els atributs de cada municipi i la llsta de partits polítics
    
    counter = 0 # S'inicia contador a 0
    #while counter < len(llista_municipis):
    while counter < 1: #Test de 2 municipis
        # Es fa la lectura de URL de cada municipi
        url = elec_2019[counter].getUrl()
        # Càrrega i obtenció de la pàgina web
        page = requests.get(url)
        soup = BeautifulSoup(page.content,"html.parser")
        
        # Obtenció de les taules en format TAG
        taula_2019 = soup.find('table')             # Partits polítics 2019
        dades_2019 = taula_2019.find_next('table')  # Informació extra 2019
        taula_2015 = dades_2019.find_next('table')  # Partits polítics 2015
        dades_2015 = taula_2015.find_next('table')  # Informació extra 2015
              
        # S'extreuen els atributs dels municipis
        lecturaAtributs(dades_2019,elec_2019[counter])
        elec_2019[counter].getAtributs()
        lecturaAtributs(dades_2015,elec_2015[counter])
        elec_2015[counter].getAtributs()
        #S'extreuen els partits politics de cada municipi
        lecturaMunicipis(taula_2019,elec_2019[counter])
        lecturaMunicipis(taula_2015,elec_2015[counter])
        
        counter += 1


# 2. Descarrega de les dades
Es descarrega la informació del lloc web d'interes, en aquest cas de les eleccions del 2015 i 2019 per municipis a catalunya.


In [6]:
# Carrega de llibreries necessaries
import requests
from bs4 import BeautifulSoup

In [7]:
# Es defineix la web de recerca
web = "https://www.naciodigital.cat/municipals2019"

# Es defineix les llistes de municipis buides. Els municipis són els mateixos per 2015 i 2019.
elec_2019 = []
elec_2015 = []

# Es crida la funció llistaMunicipis()
llistaMunicipis(elec_2019,elec_2015,web)

# S'extreuen les dades
extraccioDades(elec_2019,elec_2015)

Nom:  Abella de la Conca  Cens:  146  Escrutat:  100,00% Abstencio:  24 Vots comptabilitats:  122  Vots nuls:  0  Vots blancs:  0
Nom:  Abella de la Conca  Cens:  140  Escrutat:  100,00% Abstencio:  32 Vots comptabilitats:  108  Vots nuls:  3  Vots blancs:  0


In [8]:
elec_2019[0].partits[0].getTot()


Nom:   ERC-AM  Vots:  64  %:  52,46%  Regidos:  4


In [9]:
elec_2015[0].partits[0].getTot()

Nom:   JIAB - AM  Vots:  53  %:  50,48%  Regidos:  4
